In [1]:
import pandas as pd
import json
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import math
import numpy as np
import pandas as pd
import nltk
import spacy
from tqdm import tqdm
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

In [2]:
all_data=pd.read_csv('WikiLarge_Train.csv')

In [3]:
nlp = spacy.load('en_core_web_sm')
tokenizer = RegexpTokenizer(r'\w+')
def tokenizer(text):
    return tokenizer.tokenize(text)
  
def remove_stopwords(review):
    all_word = []
    for sentence in tqdm(review):
        all_word_list = []
        for token in sentence:
            if nlp.vocab[token].is_stop == True:
                continue
            else:
                all_word_list.append(token)
        all_word.append(" ".join(all_word_list))    
        
    return all_word

In [11]:
from nltk.tokenize import RegexpTokenizer
rest_tokenizer = RegexpTokenizer(r'\w+')
def tokenizer(text):
    return rest_tokenizer.tokenize(text)

In [12]:

def result_evalation(predict_result, true_result):
    print("accuracy:", accuracy_score(true_result, predict_result))
    print("precision:", precision_score(true_result, predict_result))
    print("recall:", recall_score(true_result, predict_result))
    print("f1:", f1_score(true_result, predict_result))


In [14]:
tokenized_text=[]
for i in range(len(all_data)):
    tokenized_text.append(tokenizer(all_data['original_text'][i]))
tokenized_text_new=remove_stopwords(tokenized_text)

100%|██████████| 416768/416768 [00:08<00:00, 47889.75it/s]


In [18]:
all_data['tokenize']=tokenized_text_new

In [19]:
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(all_data, all_data['label']):
    strat_train_set=all_data.iloc[train_index]
    strat_dev_set=all_data.iloc[test_index]
strat_dev_set['label'].value_counts()/len(strat_dev_set)

1    0.5
0    0.5
Name: label, dtype: float64

In [22]:
test_data=pd.read_csv('WikiLarge_Test.csv')

In [24]:
test_data

,id,original_text,label
0,0,-2011,NaN
1,1,-2011,NaN
2,2,-2000,NaN
3,3,-1997,NaN
4,4,1.636,NaN
...,...,...,...
119087,119087,#NAME?,NaN
119088,119088,#NAME?,NaN
119089,119089,#NAME?,NaN
119090,119090,#NAME?,NaN


In [20]:
strat_train_set.reset_index(inplace=True)
strat_dev_set.reset_index(inplace=True)

In [62]:
train_word_is_simple=[]
train_word_is_not_simple=[]
train_label=strat_train_set['label'].tolist()
train_text_no_punc=strat_train_set['tokenize'].tolist()
dev_label=strat_dev_set['label'].tolist()
dev_text_no_punc=strat_dev_set['tokenize'].tolist()

In [63]:
for i in range(len(strat_train_set)):
    if int(train_label[i])==1:
        train_word_is_not_simple.extend(tokenizer(train_text_no_punc[i]))
    else:
        train_word_is_simple.extend(tokenizer(train_text_no_punc[i]))

In [64]:
train_word_size=len(set(train_word_is_not_simple+train_word_is_simple))

In [65]:
p_y1=train_label.count(1)/len(train_label)
p_y0=train_label.count(0) / len(train_label)

In [66]:
from collections import Counter
train_text_word_counts_not_simple=Counter(train_word_is_not_simple)

train_text_word_counts_simple=Counter(train_word_is_simple)

In [71]:
def train(y_j,
          x_i,
          smoothing_alpha=1):

    smoothing_additive = smoothing_alpha * train_word_size
    if int(y_j) == 1:
        try:
            return (train_text_word_counts_not_simple[x_i] + smoothing_alpha
                ) / (len(train_word_is_not_simple) + smoothing_additive)
        except:
            return (( smoothing_alpha
                ) / (len(train_word_is_not_simple) + smoothing_additive))
            
    if int(y_j) == 0:
        try :
            return (train_text_word_counts_simple[x_i] + smoothing_alpha) / (
            len(train_word_is_simple) + smoothing_additive)
        except:
            return (  smoothing_alpha) / (
            len(train_word_is_simple) + smoothing_additive)

In [72]:
def classify(word_list,smoothing_alpha):
    word_seperated=tokenizer(word_list)
    word_seperated_no_stop=word_seperated
    #for word in word_seperated:
     #   if word not in STOP_WORDS:
      #      word_seperated_no_stop.append(word)
    p_x_y0=np.log(p_y0)
   # print(p_x_y0)
    p_x_y1=np.log(p_y1)
   # print(p_x_y1)
    for i in range(len(word_seperated_no_stop)):
        temp1=train(0,word_seperated_no_stop[i],smoothing_alpha)
        #print(temp1)
        temp2=train(1,word_seperated_no_stop[i],smoothing_alpha)
        #print(temp2)
        if temp1==0 and temp2!=0:
            #print("yes")
            return 1
        if temp2==0 and temp1!=0:
            #print("no")
            return 0
        if temp1!=0 and temp2!=0:
            p_x_y0=p_x_y0+np.log(temp1)
            p_x_y1=p_x_y1+np.log(temp2)
        if temp1==0 and temp2==0:
            # both numerators are 0
            return np.random.binomial(1,probability,1)[0]
    if(p_x_y0>p_x_y1):
        return 0
    else:
        return 1
   

In [73]:
predict_result_dev=[]
for smoothing_alpha in [1]:
    for i in range(len(dev_text_no_punc)):
        temp=classify(dev_text_no_punc[i],smoothing_alpha)
        predict_result_dev.append(temp)


In [74]:
result_evalation(predict_result_dev,test_label)

accuracy: 0.5762290951843942
precision: 0.574069756597967
recall: 0.5908054802409003
f1: 0.5823173976279724


In [75]:
tokenized_text_test=[]
for i in range(len(test_data)):
    tokenized_text_test.append(tokenizer(test_data['original_text'][i]))
tokenized_text_test=remove_stopwords(tokenized_text_test)

100%|██████████| 119092/119092 [00:01<00:00, 83679.57it/s]


In [76]:
predict_result_test=[]
for smoothing_alpha in [1]:
    for i in range(len(tokenized_text_test)):
        temp=classify(tokenized_text_test[i],smoothing_alpha)
        predict_result_test.append(temp)


In [77]:
result_1=pd.DataFrame({'id':list(test_data['id']), 'label':predict_result_test})
result_1.to_csv('results_NB.csv',index=None)